# AutoML with hyperopt-sklearn

## Use the diabetes dataset

In [1]:
from sklearn import datasets
import pandas as pd
diabetes = datasets.load_diabetes()
dia_df = pd.DataFrame(data=diabetes.data, columns=diabetes.feature_names)
dia_df.columns = ['age',
  'sex',
  'bmi',
  'bp',
  'tc',
  'ldl',
  'hdl',
  'tch',
  'ltg',
  'glu']
dia_df["target"] = diabetes.target

## Train test split

In [2]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(dia_df, train_size=0.8, test_size=0.2, random_state=1231)

## hyperopt-sklearn autoML workflow

In [3]:
from hpsklearn import HyperoptEstimator
from hpsklearn import any_regressor
from hpsklearn import any_preprocessing
from hyperopt import tpe

# define search use tpe suggested algorithm
model = HyperoptEstimator(regressor=any_regressor('rgr'), preprocessing=any_preprocessing('pre'), algo=tpe.suggest, max_evals=50, trial_timeout=30)
# perform the search
model.fit(df_train.iloc[:,0:10], df_train['target'])

 94%|████████████████████████████████████████████████████████████████████████████▉     | 15/16 [00:00<?, ?trial/s, best loss=?]

/Users/zhangz5q/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


100%|█████████████████████████████████████████████████████████| 39/39 [00:02<00:00,  2.45s/trial, best loss: 0.611030297522025]

/Users/zhangz5q/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(



100%|████████████████████████████████████████████████████████| 49/49 [00:02<00:00,  2.50s/trial, best loss: 0.6041518428725485]

/Users/zhangz5q/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(



100%|████████████████████████████████████████████████████████| 50/50 [00:02<00:00,  2.51s/trial, best loss: 0.6041518428725485]


## Performance on the test data

In [5]:
model.score(df_test.iloc[:,0:10], df_test['target'])

/Users/zhangz5q/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(


0.5259070668921988

In [9]:
model.predict(df_test.iloc[1:3,0:10])

/Users/zhangz5q/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(


array([230.84881978, 199.79898839])

## Check the best model

In [6]:
print(model.best_model())

{'learner': RandomForestRegressor(criterion='absolute_error', max_depth=4,
                      max_features='sqrt', max_leaf_nodes=15,
                      min_impurity_decrease=0.02, min_samples_split=3,
                      n_estimators=2669, n_jobs=1, random_state=0,
                      verbose=False), 'preprocs': (Normalizer(),), 'ex_preprocs': ()}


A random forest model seems to be the best model 
using the hyperopt_sklearn autoML approach

## save the model

In [7]:
import pickle
filehandler = open(b"diabetes_hyperopt_automl.model","wb")
pickle.dump(model,filehandler)
filehandler.close()

## Use R2 to score the predictions

In [10]:
from sklearn.metrics import r2_score
y_test_predicted_hyperopt=model.predict(df_test.iloc[:,0:10])
r2_score(df_test["target"],y_test_predicted_hyperopt)

/Users/zhangz5q/Library/Python/3.8/lib/python/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(


0.5259070668921988

## Use mean absolute error to score the predictions

In [11]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(df_test["target"],y_test_predicted_hyperopt)

44.89379517641165